In [1]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot
import numpy

AMUSE pre-defines a number of calculcated attributes on particle sets, such as the kinetic energy of the particles in the set. These calculated attributes are used often and provide a sufficient set to start out with, but they do not define a *complete* set. It's possible to define your own attributes and extend the attributes on a particle set.

In [2]:
from amuse.ic.kingmodel import new_king_model
from amuse.units import nbody_system

As shown in the previous example, you can create a particle set by specifying the number of particles and setting their attributes. You can also create a particle set by using an inital condition function. For stellar clusters the commonly used plummer and king models are available. For this tutorial we will start with a king model. Global clusters created with a king model need the number of stars in the cluster and a dimensionless depth parameter that determines the depth of the potential well in the center of the cluster.

In [3]:
particles = new_king_model(1000, 3)
print(particles)

                 key         mass       radius           vx           vy           vz            x            y            z
                   -         mass       length  length / time  length / time  length / time       length       length       length
====================  ===========  ===========  ===========  ===========  ===========  ===========  ===========  ===========
16152840208084452440    1.000e-03    0.000e+00   -2.839e-01    3.127e-01    4.290e-01   -6.924e-01   -8.056e-01   -1.037e+00
14431881290045695356    1.000e-03    0.000e+00   -1.766e-01    1.722e-02   -5.473e-01   -1.608e-01    2.406e-01   -6.052e-01
11587877815567512489    1.000e-03    0.000e+00    5.826e-01   -4.158e-01    5.093e-01   -3.430e-01    4.329e-02    6.916e-01
 6105450060604620649    1.000e-03    0.000e+00   -2.974e-01   -2.402e-01    6.086e-01    1.057e+00    1.026e+00   -7.428e-02
10568920569270638365    1.000e-03    0.000e+00   -4.624e-01    4.292e-01   -1.580e-01    2.631e-01    8.663e-01   -5.30

/Users/rieder/Code/amuse/src/amuse/datamodel/particles.py:286: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if numpy.issubdtype(quantity.dtype, numpy.float):


Common properties for a stellar cluster are its  center of mass position, total kinetic energy and potential energy.

In [4]:
print("center of mass", particles.center_of_mass())
print("kinetic energy", particles.kinetic_energy())
print("potential energy", particles.potential_energy(G = nbody_system.G))

center of mass [-7.8062556419e-18, 0.0, -6.93889390391e-18] length
kinetic energy 0.253613861612 length**2 * time**-2 * mass
potential energy -0.503391406401 length**2 * time**-2 * mass


For the potential energy calculation we need to specify the gravitational constant, as the default value will use the gavitational constant in S.I. units and we are working in nbody units for this tutorial.

In N-body calculations and reporting, the kinetic and potential energy of a set of stars is often scaled to exactly 0.25 and -0.5 respectively. AMUSE also has a function for this.

In [5]:
particles.scale_to_standard()
print("kinetic energy", particles.kinetic_energy())
print("potential energy", particles.potential_energy(G = nbody_system.G))

kinetic energy 0.25 length**2 * time**-2 * mass
potential energy -0.5 length**2 * time**-2 * mass


*Note that the potential energy and scaling calculations are implemented as order N-squared operations*

Attributes of particle sets are always 1 dimensional by default, an array with a single value per particle attribute. But for some attributes it is easier to work with a 2d set, an array with multiple values (or an array of values) per particle attribute. For example, the positions of all particles. These attributes are called vector-attributes and are defined as a combination of 2 or more simple attributes. 

The position attribute combines the values of the `x`, `y` and `z` attributes.

In [6]:
print(particles[0].x)
print(particles[0].y)
print(particles[0].z)
print(particles[0].position)

-0.697145382413 length
-0.811089903872 length
-1.04439489954 length
[-0.697145382413, -0.811089903872, -1.04439489954] length


Other common vector attributes are `velocity` (combination of `vx`,`vy`,`vz`) and `acceleration` (combination of `ax`,`ay`,`az`).

You can set the value of a position attribute and the underlying x, y or z attributes will be changed. 

In [7]:
particles[0].position = [0, 0.1, 0.2] | nbody_system.length
print(particles[0].x)
print(particles[0].y)
print(particles[0].z)

0.0 length
0.1 length
0.2 length


You can set the value of the x, y or z attribute and the position will change (as the position is just a combination of these attributes).

In [8]:
particles[0].x = 0.3 | nbody_system.length
print(particles[0].position)

[0.3, 0.1, 0.2] length


You cannot change an item in the position array and thereby change the x, y, or z positions

In [9]:
particles[0].position[0] = 0.5 | nbody_system.length # this will not change anything in the particles set as the position is a copy
print(particles[0].x)
print(particles[0].position)

0.3 length
[0.3, 0.1, 0.2] length


You can use the position attribute on the entire set. Let's print the positions of the first 10 particles.

In [10]:
print(particles.position[0:10])

[[0.3, 0.1, 0.2], [-0.16192980401, 0.242192888358, -0.609337864298], [-0.345353468543, 0.0435869644753, 0.696307628153], [1.06411307118, 1.03279896797, -0.0747807403124], [0.264910325225, 0.872165603883, -0.533783347579], [0.157305699399, -0.228433295489, 0.606580812789], [-1.23571515683, -0.261535452729, 0.266109928239], [-0.552614729364, 0.130276728259, -0.760368454879], [0.232129372088, 0.271623802249, 1.18843899364], [-0.0608235745513, -0.303341196542, 0.477802135591]] length


You can also use the position attribute to set values for the entire set

In [11]:
particles.position = [0.1, 0.2, 0.3] | nbody_system.length # set the position of all particles in the set to the same value
print(particles.position[0:10])
print(particles.x[0:10])

[[0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3], [0.1, 0.2, 0.3]] length
[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1] length


Defining a new vector attribute is done by calling the `add_vector_attribute` or `add_global_vector_attribute`. The first call will define the attribute on the particle set and not on any other set. The second call will define the attribute on the particle set and any future sets created in the script. (The second call is used in the amuse framework itself to define the `position`, `velocity` and `acceleration` attributes)

In [12]:
particles.add_vector_attribute('position2d', ('x', 'y'))
print(particles[0].position2d)

[0.1, 0.2] length


If you enter `particles.add_` and press tab you'll notice two other function besides the `add_vector_attribute` function; `add_calculated_attribute` will create an attribute where the values are calculated based on other attributes, `add_function_attribute` will create a function on the set that gets the set and optional function parameters. These function also have global versions (`add_global_...`). The `add_global_function_attribute` call is used in the AMUSE framework to implement the `kinetic_energy` and `potential_energy` functions.


In [13]:
particles.add_function_attribute('calculate_total_mass', lambda particles : particles.mass.sum())
print(particles.calculate_total_mass())

1.0 mass
